### Summary:
The main difference between the implementation of DeepWalk in Java and Python is their use of data structures to define graphs and cache inputs for the embedding calculations.
+ Most of the functionality is abstracted from Gensim's Word2Vec.
+ Look at the python functions unique to Python, it seems like a reasonable to port to Scala the implementations of:   
++ **Graph class** and <u>build_randomwalk_corpus</u> and <u>random_walk</u>.

In [3]:
reset -fs

In [4]:
import random
import numpy as np

import graph2 as graph
from gensim.models import Word2Vec
from skipgram import Skipgram

## Example of the input document

In [5]:
! cat example_graphs/p2p-Gnutella08.edgelist | head -n 11












cat: stdout: Broken pipe


> Edgelist that contains start node and end node per line

## Graph -> Load Edgelist

In [6]:
G = graph.load_edgelist('example_graphs/p2p-Gnutella08.edgelist', undirected=True)

+ Load edgelist:
    Builds a list out of node connectionsm

In [7]:
# Display connections to edge 0 as seen above
G[0]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [8]:
G[80]

[76, 531, 2014]

In [6]:
print("Number of nodes: {}".format(len(G.nodes())))

Number of nodes: 6301


+ "build_deepwalk_corpus" uses "random_walk" to create a list of lists.       
For each node it takes **N walks** based on **"num_paths"** each of the walks is **"path_lenghts"** long.

## Graph > Build_deepwalk_corpus

In [9]:
walks = graph.build_deepwalk_corpus(G, 
                                    num_paths=10,
                                    path_length=10)

``` python
def build_deepwalk_corpus(G, num_paths, path_length, alpha=0,
                      rand=random.Random(0)):
  walks = []

  nodes = list(G.nodes())
  
  for cnt in range(num_paths):
    rand.shuffle(nodes)
    for node in nodes:
      walks.append(G.random_walk(path_length, rand=rand, alpha=alpha, start=node))
  
  return walks
```

In [10]:
walks

[[1826, 1821, 1636, 2629, 3102, 4536, 3102, 1066, 1057, 1251],
 [1297, 1864, 1608, 3014, 4458, 541, 1529, 3333, 1529, 3330],
 [2524, 5840, 2524, 1252, 1251, 1255, 5915, 2161, 5915, 2161],
 [3797, 3600, 1855, 3600, 3797, 788, 787, 792, 787, 794],
 [3563, 1819, 2869, 1819, 3565, 2343, 1766, 1143, 2384, 4070],
 [2506, 544, 2508, 544, 2511, 544, 2512, 544, 5207, 4312],
 [5900, 5615, 5900, 5615, 4854, 3997, 3702, 5804, 5995, 122],
 [5361, 5789, 5361, 5789, 5361, 5352, 3614, 3568, 4837, 3568],
 [5782, 5343, 2414, 2936, 4412, 2936, 4414, 4282, 2718, 3558],
 [4792, 2538, 4792, 4199, 4792, 3514, 4792, 874, 2762, 3858],
 [1353, 572, 4839, 3441, 4841, 3881, 3488, 2614, 4214, 2614],
 [252, 9, 371, 4, 352, 3587, 4455, 4569, 2965, 4569],
 [5341, 5899, 17, 3893, 147, 314, 147, 3893, 252, 9],
 [198, 2148, 3134, 5304, 2440, 448, 108, 2425, 4100, 2425],
 [3032, 1694, 6171, 1022, 2743, 14, 2743, 1278, 412, 2532],
 [2927, 1749, 225, 893, 734, 893, 5113, 1590, 1585, 1588],
 [4893, 3703, 3204, 1814, 3550, 2

In [11]:
len(walks)

63010

> We had 6310 nodes and selected 10 walks to end up with 63010

## Graph > Build_deepwalk_corpus > random_walk

The implementation of tha random_walk below:


``` python
  def random_walk(self, path_length, alpha=0, rand=random.Random(), start=None):
    """ Returns a truncated random walk.

        path_length: Length of the random walk.
        alpha: probability of restarts.
        start: the start node of the random walk.
    """
    G = self
    if start:
      path = [start]
    else:
      # Sampling is uniform w.r.t V, and not w.r.t E
      path = [rand.choice(list(G.keys()))]

    while len(path) < path_length:
      cur = path[-1]
      if len(G[cur]) > 0:
        if rand.random() >= alpha:
          path.append(rand.choice(G[cur]))
        else:
          path.append(path[0])
      else:
        break
    return path
```

## Gensim's Word2Vec

The random walks are passed to Gensim's implementation of Word2vec, taking in consideration **'size of embeddings'**, **'window'** and **'workers'**

> The rest of the functionality comes from Gensim

In [9]:
# http://stackoverflow.com/questions/17322668/typeerror-dict-keys-object-does-not-support-indexing

In [10]:
# model = Word2Vec(walks, size=args.representation_size, window=args.window_size, min_count=0, workers=args.workers)
model = Word2Vec(walks, size=20, window=2, min_count=0, workers=1)

/Users/jaimealmeida/anaconda/lib/python3.6/site-packages/gensim/models/word2vec.py:406: UserWarning: Cython compilation failed, training will be slow. Do you have Cython installed? `pip install cython`
  warnings.warn("Cython compilation failed, training will be slow. Do you have Cython installed? `pip install cython`")


In [11]:
# save embeddings
model.save_word2vec_format('p2p-Gnutella08.embeddings')

In [17]:
# this should be the size of the embeddings
model.layer1_size

20

In [12]:
# to access embedding of node
model[6139]

array([ 0.99090689,  1.57357669, -0.74200916,  0.00366741, -0.59294271,
        0.76989913, -0.08553434,  0.23412816,  1.05244756, -1.54374409,
        0.21923834, -1.65432942, -0.31502897, -0.24664499, -0.29436067,
        0.20422186, -0.40994814, -0.5568487 ,  0.2800861 , -0.66820282], dtype=float32)

In [27]:
user6139 = model[6139]

In [31]:
# to find topn users similar
model.most_similar(positive=[user6139], topn=5)

[(6139, 0.9999998807907104),
 (6204, 0.847969651222229),
 (6214, 0.8393740653991699),
 (6213, 0.8295308351516724),
 (6197, 0.8277329802513123)]

# MovieLens 1M data set

In [19]:
! cat example_graphs/ml_threshold.csv | head -n 11

289	3638
393	3686
5953	3112
471	3117
2451	5828
5704	5418
5421	2630
712	4834
669	4987
4321	4223
5735	1655
cat: stdout: Broken pipe


In [ ]:
movielens_graph = graph.load_edgelist('example_graphs/ml_threshold.csv', undirected=True)

In [14]:
print("Number of nodes: {}".format(len(movielens_graph.nodes())))

Number of nodes: 5639


In [15]:
movielens_walks = graph.build_deepwalk_corpus(movielens_graph, 
                                    num_paths=10,
                                    path_length=10)

In [16]:
movielens_walks

[[4597, 459, 1934, 4074, 116, 2081, 979, 3697, 2955, 696],
 [2354, 2463, 638, 830, 1066, 2525, 5861, 4104, 2180, 5958],
 [2461, 4283, 4078, 1390, 4268, 2922, 2730, 2087, 458, 5523],
 [3338, 1628, 3057, 4785, 3894, 3889, 4909, 4051, 4660, 3555],
 [4243, 4478, 1583, 4600, 194, 4498, 4235, 3830, 4435, 378],
 [2695, 5816, 429, 4548, 5277, 940, 3381, 1142, 783, 4094],
 [3077, 2132, 5242, 590, 1413, 670, 5733, 5353, 4022, 2171],
 [5199, 2541, 4438, 5684, 4600, 1160, 5940, 1055, 3574, 5032],
 [2148, 4498, 1879, 5489, 3210, 5912, 4031, 667, 2093, 2530],
 [3106, 937, 4032, 4490, 2407, 2059, 2874, 1021, 2088, 1529],
 [4524, 1225, 2149, 1423, 5286, 5834, 1543, 5840, 346, 382],
 [3234, 3371, 5160, 2898, 3876, 4495, 5140, 2250, 1686, 2632],
 [5989, 5762, 4392, 2694, 427, 822, 5620, 2253, 3858, 1331],
 [1817, 150, 3150, 3058, 4347, 1353, 235, 5467, 2901, 4680],
 [5671, 480, 5293, 1694, 5028, 1230, 5453, 2876, 1091, 4212],
 [639, 81, 1752, 2489, 5378, 2547, 4762, 5722, 3487, 5275],
 [3360, 5091, 2763

In [17]:
len(movielens_walks)

56390

In [18]:
model = Word2Vec(movielens_walks, size=20, window=2, min_count=0, workers=1)

/Users/jaimealmeida/anaconda/lib/python3.6/site-packages/gensim/models/word2vec.py:406: UserWarning: Cython compilation failed, training will be slow. Do you have Cython installed? `pip install cython`
  warnings.warn("Cython compilation failed, training will be slow. Do you have Cython installed? `pip install cython`")


In [35]:
model.save_word2vec_format('movielens.embeddings')

In [26]:
user289 = model[289]

In [29]:
user2438 = model[2438]

In [27]:
# to find topn users similar
model.most_similar(positive=[user289], topn=5)

[(289, 1.0),
 (2438, 0.966269314289093),
 (902, 0.9543932676315308),
 (1065, 0.9533358216285706),
 (4757, 0.9518369436264038)]

In [30]:
model.most_similar(positive=[user2438], topn=5)

[(2438, 1.0),
 (902, 0.9760292768478394),
 (3938, 0.9730705618858337),
 (5902, 0.9726033210754395),
 (3869, 0.9725024700164795)]

In [32]:
# finding topn users similar to two provided users
model.most_similar(positive=[user289, user2438], topn=5)

[(2438, 0.993671178817749),
 (289, 0.9890822172164917),
 (902, 0.9747748374938965),
 (5902, 0.9696666598320007),
 (1065, 0.9682540893554688)]

## Next Step:

+ We can impute the rating for a movie for **user 289** based on the weighted average (LLR score) rating of that movie for the nearest users who rated that movie.